In [2]:
%autosave 60

Autosaving every 60 seconds


In [35]:
import re
import os
import sys
import time
import requests
import datetime
import numpy as np
import pandas as pd
from dateutil.relativedelta import *
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [4]:
months = dict(zip(['августа',
 'апреля',  'декабря',  'июля',
 'июня',  'марта',  'мая',
 'ноября',  'октября',  'сентября',
 'февраля',  'января'], [8, 4, 12, 7, 6, 3, 5, 11, 10, 9, 2, 1]))

In [5]:
s=Service(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\User\.wdm\drivers\chromedriver\win32\98.0.4758.102]


In [7]:
# !"{sys.executable}" -m pip install --upgrade webdriver_manager

In [8]:
# # Read already gathered data

# df_first_articles = pd.DataFrame()
# for i in os.listdir(os.getcwd()):
#     if i.find(".csv") > -1:
#         df_temp = pd.read_csv(i, sep=";")        
#         df_first_articles = pd.concat([df_first_articles, df_temp], sort=True)
# df_first_articles.columns=["num", "datestr", "type", "header", "text", "url"]
# df_first_articles = df_first_articles.sort_values(by=['num']).reset_index(drop=True)

## Часть 1: парсим новый сайт ЦБ
### Выгрузка пресс-релизов, новостей и выступлений
https://www.cbr.ru/

In [105]:
# Create two windows: one with the main site and one for sub-windows

url = "https://www.cbr.ru/"

driver_base = webdriver.Chrome(service=s)
driver_base.maximize_window()
driver_base.get(url)
parent_tab = driver_base.window_handles[0]

driver_runner1 = webdriver.Chrome(service=s)
driver_runner1.maximize_window()

In [106]:
# Собираем в 4 этапа: 
# Новости
# Пресс-релизы
# Интервью - не нужно
# Выступления

# Так как приходится кликать разделы вручную :(
# Запустить кликер -> считать категорию -> обновить страницу -> повторить

In [110]:
def parse_by_type(articles_headers, articles_links):
    counter = 0
    articles = []
    exception_articles = []
    for header, link in tqdm(zip(articles_headers, articles_links)):
        counter += 1
        try:
            article_type = header.text.split("\n")[-1]
            article_url = link.get_attribute(name='href')
            driver_runner1.get(article_url)
            article_date = driver_runner1.find_elements(By.TAG_NAME, value='.news-info-line_date')[0].text
            article_text = [a.text for a in driver_runner1.find_elements(By.TAG_NAME, value='p')]
            articles.append([counter, article_date, article_type, link.text, "\n".join(article_text).strip(), article_url])
        except Exception as e:
            print("Problem!")
            exception_articles.append((header, link))  
    return articles, exception_articles

In [207]:
%%time

# Clicker!
# Click all buttons "Загрузить ещё" to get all the news in one window

try:
    while True:
        buttons = driver_base.find_elements(By.TAG_NAME, value='#_buttonLoadNextEvt')
        button = [i for i in buttons if i.text == 'Загрузить еще']
        button[0].click()
except Exception as e:
    print(e)
    print("End reached!")

list index out of range
End reached!
Wall time: 1.44 s


In [108]:
news_headers = driver_base.find_elements(By.TAG_NAME, value='.news_info')
news_links = driver_base.find_elements(By.TAG_NAME, value='.news_title')
print(len(news_headers), len(news_links))

1020 1020


In [111]:
news_articles, news_exc = parse_by_type(news_headers, news_links)

df_news_new = pd.DataFrame(news_articles)
df_news_new = df_news_new[df_news_new[2] != ""].reset_index(drop=True)
df_news_new[0] = df_news_new[0] - 20

1020it [13:22,  1.27it/s]


In [153]:
pr_headers = driver_base.find_elements(By.TAG_NAME, value='.news_info')
pr_links = driver_base.find_elements(By.TAG_NAME, value='.news_title')
print(len(pr_headers), len(pr_links))

1020 1020


In [201]:
pr_articles, pr_exc = parse_by_type(pr_headers, pr_links)

df_pr_new = pd.DataFrame(pr_articles)
df_pr_new = df_pr_new[df_pr_new[2] != ""].reset_index(drop=True)
df_pr_new[0] = df_pr_new[0] - 20

In [205]:
a = df_pr_new.reset_index().iloc[720:]
a[a['index'] != a[0] -2]
df_pr_new.iloc[740:760]
# header, link = pr_exc[1]

# article_type = header.text.split("\n")[-1]
# article_url = link.get_attribute(name='href')
# driver_runner1.get(article_url)
# # article_date = driver_runner1.find_elements(By.TAG_NAME, value='.news-info-line_date')[0].text
# article_text = [a.text for a in driver_runner1.find_elements(By.TAG_NAME, value='p')]
# article_date = "11 марта 2020 года"

# df_pr_new = df_pr_new.append([[760, article_date, article_type, link.text, "\n".join(article_text).strip(), article_url]])
# df_pr_new = df_pr_new.sort_values(by=[0]).reset_index(drop=True)

In [208]:
speech_headers = driver_base.find_elements(By.TAG_NAME, value='.news_info')
speech_links = driver_base.find_elements(By.TAG_NAME, value='.news_title')
print(len(speech_headers), len(speech_links))

153 153


In [209]:
speech_articles, speech_exc = parse_by_type(speech_headers, speech_links)

153it [05:55,  2.32s/it]


In [213]:
df_speech_new = pd.DataFrame(speech_articles)
df_speech_new = df_speech_new[df_speech_new[2] != ""].reset_index(drop=True)
df_speech_new[0] = df_speech_new[0] - 20

In [ ]:
df_new = pd.DataFrame()
for df_articles_temp, name in zip([df_news_new, df_pr_new, df_speech_new], ["news","press_rel","speech"]):
    df_articles_temp.to_csv(f"{name}_new.csv", 
                            encoding="utf-8", sep=";", index=False)
    df_new = pd.concat([df_new, df_articles_temp])
df_new.columns=["num", "datestr", "type", "header", "text", "url"]
df_new = df_new.drop(columns=['num'])
df_new = df_new.reset_index(drop=True)

### Здесь можно забрать готовые данные

In [9]:
df_new = pd.DataFrame()
for name in ["news","press_rel","speech"]:
    df_articles_temp = pd.read_csv(f"{name}_new.csv", 
                            encoding="utf-8", sep=";")
    df_new = pd.concat([df_new, df_articles_temp])
df_new.columns=["num", "datestr", "type", "header", "text", "url"]
df_new = df_new.drop(columns=['num'])
df_new = df_new.reset_index(drop=True)

df_new['date'] = [f"{i[0]}-{months[i[1]]}-{i[2]}" for i in df_new.datestr.str.split().tolist()]
df_new['date'] = pd.to_datetime(df_new['date'], format='%d-%m-%Y')

In [10]:
df_new.head()

,datestr,type,header,text,url,date
0,21 февраля 2022 года,Новость,Объем средств на счетах эскроу в долевом строи...,"Объем средств, размещенных участниками долевог...",https://www.cbr.ru/press/event/?id=12705,2022-02-21
1,17 февраля 2022 года,Новость,Мониторинг отраслевых финансовых потоков: рост...,Динамика отраслевых финансовых потоков указыва...,https://www.cbr.ru/press/event/?id=12702,2022-02-17
2,17 февраля 2022 года,Новость,В январе инфляция ускорилась в большинстве рег...,Годовая инфляция в январе находилась в диапазо...,https://www.cbr.ru/press/event/?id=12701,2022-02-17
3,17 февраля 2022 года,Новость,Банк России разработал детскую игру о признака...,Игра «Космический детектив: тайна семи банкнот...,https://www.cbr.ru/press/event/?id=12698,2022-02-17
4,16 февраля 2022 года,Новость,"В 2021 году Банк России выявил почти 2,7 тысяч...",Среди них 871 финансовая пирамида (в 2020 году...,https://www.cbr.ru/press/event/?id=12695,2022-02-16


In [11]:
df_new[df_new.type == "Пресс-релиз"].min()

datestr                                  1 августа 2019 года
type                                             Пресс-релиз
header     А. Данилов назначен директором Департамента об...
text       1. В целях применения абзаца второго и третьег...
url        https://www.cbr.ru/press/pr/?file=01022021_084...
date                                     2019-07-17 00:00:00
dtype: object

## Часть 2: парсим старый сайт ЦБ
### Выгрузка пресс-релизов, новостей и выступлений
http://old.cbr.ru/press/

In [48]:
url = "http://old.cbr.ru/press/month_archive/press_archive/"

driver_base = webdriver.Chrome(service=s)
driver_base.maximize_window()
driver_base.get(url)
parent_tab = driver_base.window_handles[0]

driver_runner1 = webdriver.Chrome(service=s)
driver_runner1.maximize_window()

In [49]:
start_date = datetime.date(2010, 1, 1)
end_date = datetime.date(2019, 7, 16)

In [51]:
DETECTED = df.url.tolist() 

def half_parser(cur_date, end_date, counter):
    exc_articles = []
    good_articles = []
    
    driver_base.find_elements(By.TAG_NAME, value='.hasDatepicker')[0].send_keys("\b\b\b\b\b\b\b\b\b\b" + cur_date.strftime("%d.%m.%Y"))
    driver_base.find_elements(By.TAG_NAME, value='.hasDatepicker')[1].send_keys("\b\b\b\b\b\b\b\b\b\b" + end_date.strftime("%d.%m.%Y"))
    driver_base.find_elements(By.TAG_NAME, value='#UniDbQuery_searchbutton')[0].click()

    articles_dates = driver_base.find_elements(By.TAG_NAME, value='.source')
    articles_links = driver_base.find_elements(By.TAG_NAME, value='a')[15:-10]
    for article_date, link in tqdm(zip(articles_dates, articles_links)):
        if link.get_attribute(name='href') in DETECTED:
            continue
        counter += 1
        try:
            article_type = "Пресс-релиз"
            article_url = link.get_attribute(name='href')
            driver_runner1.get(article_url)
            article_text = [a.text for a in driver_runner1.find_elements(By.TAG_NAME, value='p')]
            good_articles.append([counter, article_date.text, article_type, link.text, "\n".join(article_text).strip(), article_url])
        except Exception as e:
            print("Problem!")
            exc_articles.append((article_date, link)) 
    return good_articles, exc_articles, counter


counter = 0
cur_date = start_date
temp_end = cur_date + relativedelta(months=+1) - relativedelta(days=+1)
exc_articles = []
good_articles = []
while temp_end < end_date:
    print(cur_date, " - ", temp_end)
    
    good_articles_temp, exc_articles_temp, counter = half_parser(cur_date, cur_date + relativedelta(days=+6), counter)
    good_articles += good_articles_temp
    exc_articles += exc_articles_temp
    
    good_articles_temp, exc_articles_temp, counter = half_parser(cur_date + relativedelta(days=+7), 
                                                                 cur_date + relativedelta(days=+13), counter)
    good_articles += good_articles_temp
    exc_articles += exc_articles_temp

    good_articles_temp, exc_articles_temp, counter = half_parser(cur_date + relativedelta(days=+14), 
                                                                 cur_date + relativedelta(days=+20), counter)
    good_articles += good_articles_temp
    exc_articles += exc_articles_temp
    
    good_articles_temp, exc_articles_temp, counter = half_parser(cur_date + relativedelta(days=+21), 
                                                                 cur_date + relativedelta(days=+23), counter)
    good_articles += good_articles_temp
    exc_articles += exc_articles_temp
    
    good_articles_temp, exc_articles_temp, counter = half_parser(cur_date + relativedelta(days=+24), temp_end, counter)
    good_articles += good_articles_temp
    exc_articles += exc_articles_temp

    if cur_date.month == 2:
        pd.DataFrame(good_articles).to_csv(f"{cur_date}_old_truely_all_arts.csv", encoding="utf-8", sep=";", index=False)    

    cur_date = cur_date + relativedelta(months=+1)
    temp_end = cur_date + relativedelta(months=+1) - relativedelta(days=+1)
    temp_end = min([temp_end, end_date])

2010-01-01  -  2010-01-31


0it [00:00, ?it/s]
15it [00:00, 98.01it/s]
29it [00:00, 101.56it/s]
5it [00:00, 102.82it/s]
26it [00:00, 106.28it/s]


2010-02-01  -  2010-02-28


28it [00:00, 99.64it/s]
29it [00:00, 101.26it/s]
26it [00:00, 107.92it/s]
5it [00:00, 87.72it/s]
24it [00:00, 100.62it/s]


2010-03-01  -  2010-03-31


...

8it [00:00, 111.11it/s]
6it [00:00, 103.45it/s]
9it [00:00, 104.73it/s]
6it [00:00, 125.38it/s]
10it [00:00, 113.92it/s]


2019-05-01  -  2019-05-31


5it [00:00, 86.25it/s]
1it [00:00, 111.16it/s]
6it [00:00, 96.77it/s]
7it [00:00, 79.54it/s]
6it [00:00, 93.74it/s]


2019-06-01  -  2019-06-30


9it [00:00, 56.43it/s]
10it [00:00, 116.28it/s]
7it [00:00, 98.59it/s]
0it [00:00, ?it/s]
14it [00:00, 109.37it/s]


In [53]:
pd.DataFrame(good_articles).to_csv(f"old_all_truely_all_arts.csv", encoding="utf-8", sep=";", index=False)    

In [57]:
df_pr_old_extra = pd.DataFrame(good_articles)
df_pr_old_extra.columns=["num", "datestr", "type", "header", "text", "url"]
df_pr_old_extra = df_pr_old_extra.drop(columns=['num'])
df_pr_old_extra = df_pr_old_extra.reset_index(drop=True)

# df_pr_old['date'] = [f"{i[0]}-{months[i[1]]}-{i[2]}" for i in df_new.datestr.str.split().tolist()]
df_pr_old_extra['date'] = pd.to_datetime(df_pr_old_extra['datestr'], format='%d.%m.%Y')
df_pr_old_extra.head()

,datestr,type,header,text,url,date
0,01.07.2014,Пресс-релиз,О продаже принадлежащих Банку России обыкновен...,В целях выполнения требования Федерального зак...,http://www.cbr.ru/press/PR/?file=01072014_1850...,2014-07-01
1,01.07.2014,Пресс-релиз,Об Указании Банка России от 23 апреля 2014 год...,Банк России издал Указание Банка России от 23 ...,http://www.cbr.ru/press/PR/?file=01072014_1836...,2014-07-01
2,17.07.2014,Пресс-релиз,О прекращении деятельности временной администр...,В связи с решением Арбитражного суда города Мо...,http://www.cbr.ru/press/PR/?file=17072014_1430...,2014-07-17
3,17.07.2014,Пресс-релиз,О решении Банка России от 16 июля 2014 года,Решение Банка России от 16 июля 2014 года:\n\n...,http://www.cbr.ru/press/PR/?file=17072014_1615...,2014-07-17
4,17.07.2014,Пресс-релиз,"Перечень кредитных организаций, в которые назн...","Уполномоченные представители Банка России, дей...",http://www.cbr.ru/press/PR/?file=17072014_2043...,2014-07-17


## Объединяем часть 1 и 2

In [130]:
# df = pd.concat([df_pr_old, df_new, df_pr_old_extra]).drop_duplicates().sort_values(by=['date', 'type']).reset_index(drop=True)
# df = df[['date','type','header','text', 'url']]
# df = df[(df.date.dt.date <= datetime.date(2022, 2, 1)) & (df.date.dt.date <= datetime.date(2022, 2, 1))]

In [70]:
df = pd.concat([df, df_pr_old_extra]).drop_duplicates().sort_values(by=['date', 'type']).reset_index(drop=True)
df = df[['date','type','header','text', 'url']]
df.to_csv("main_upd.csv", encoding="utf-8", sep=";", index=False)    

In [82]:
df = pd.read_csv("main_upd.csv", sep=";")
df1 = pd.read_csv("data1.csv")

df.date = pd.to_datetime(df.date).dt.date

df["rate_flag"] = (df.header.isin(set(df1['information.text'])) | ((df.type=="Пресс-релиз") & 
                                                  (df.header.str.find("Банк России принял") > -1) &
                                                  (df.header.str.find("ключ") > -1))) * 1
df.to_csv("main_upd1.csv", encoding="utf-8", sep=";", index=False)

# Часть 3: добавляем данные и генерим текстовые переменные

In [338]:
df = pd.read_csv("main_upd1.csv", encoding="utf-8", sep=";")
df.date = pd.to_datetime(df.date).dt.date
df = df[df.type != 'Новость']

In [339]:
# driver_base = webdriver.Chrome(service=s)
# driver_base.maximize_window()
# res = dict()
# exc = []
# for i in tqdm(df[df.text.str.len() < 200].url.index):
#     try:
#         url = df.loc[i].url
#         driver_base.get(url)
#         res[i] = driver_base.find_elements(By.TAG_NAME, value='.landing-text')[0].text
#     except Exception:
#         exc.append(i)

# for i, t in res.items():
#     if len(t) < len(df.loc[i].text):
#         res.pop(i)

for i, t in res.items():
    df.loc[i, 'text'] = t

### Выступления

In [340]:
# даты выступлений совпадают с датами пресс-релизов
df1 = df[df.type == 'Выступление']
rate_dates = df1[df1.header.str.find("Заявление Председателя Банка России") > -1].date
print(len(set(rate_dates) - set(df[df.rate_flag == 1].date)))
df.loc[df.date.isin(df[df.rate_flag == 1].date) & (df.type == 'Выступление'), "rate_flag"] = 1

0


In [348]:
df1.url

3754      https://www.cbr.ru/press/event/?id=5195
3972      https://www.cbr.ru/press/event/?id=5196
4085      https://www.cbr.ru/press/event/?id=5197
4285      https://www.cbr.ru/press/event/?id=5198
4425      https://www.cbr.ru/press/event/?id=5199
                           ...                   
13429    https://www.cbr.ru/press/event/?id=12361
13478    https://www.cbr.ru/press/event/?id=12416
13489    https://www.cbr.ru/press/event/?id=12429
13504    https://www.cbr.ru/press/event/?id=12451
13566    https://www.cbr.ru/press/event/?id=12542
Name: url, Length: 132, dtype: object

In [ ]:
# Анализ показал, что выступления надо парсить иначе

driver_base = webdriver.Chrome(service=s)
driver_base.maximize_window()
events = dict()
exc = []
for i in tqdm(df1.url.index):
    try:
        url = df1.loc[i].url
        driver_base.get(url)
        all_texts = driver_base.find_elements(By.TAG_NAME, value='.landing-text')
        events[i] = all_texts[0].text
    except Exception:
        exc.append(i)

In [ ]:
curlist = [i for i, t in events.items() if len(t) == 0]

In [390]:
# оказалось, что иногда надо брать вторый блок текста

driver_base = webdriver.Chrome(service=s)
driver_base.maximize_window()
exc = []
for i in tqdm(df1.url.index):
    if i in curlist:
        try:
            url = df1.loc[i].url
            driver_base.get(url)
            all_texts = driver_base.find_elements(By.TAG_NAME, value='.landing-text')
            events[i] = all_texts[1].text
        except Exception:
            exc.append(i)

100%|████████████████████████████████████████████████████████████████████████████████| 132/132 [10:22<00:00,  4.72s/it]


In [486]:
for i, t in events.items():
    if t is not None and ~df.loc[i].isna().any():
        if len(t) < len(df.loc[i].text):
            print(i, len(t), len(df.loc[i].text) - len(t))
            events.pop(i)
            
for i, t in events.items():
    df.loc[i, 'text'] = t 
    
# df1 = df[(df.rate_flag == 1) & (df.type == 'Выступление')]
# driver_base.get(df1.loc[11932].url)
# all_texts = driver_base.find_elements(By.TAG_NAME, value='.landing-text')

In [484]:
df.header = df.header.str.replace("«", '"').str.replace("»", '"')
df.text = df.text.str.replace("При использовании материала ссылка на Пресс-службу Банка России обязательна.", '')
df.text = df.text.str.strip()
df.text = df.text.str.replace("\n", " ")
df.text = df.text.apply(lambda x: re.sub('[ ]+', ' ', x))

df = df[df.text.str.len() > 130]
df.isna().any()

date         False
type         False
header       False
text         False
url          False
rate_flag    False
dtype: bool

In [492]:
df.to_csv("main_upd2.csv", encoding="utf-8", sep=";", index=False)